In [1]:
# 假设我们想对函数y = 2*x^T*x 关于列向量x求导
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
# 在我们计算y关于x的梯度之前，需要一个地方来存储梯度
x.requires_grad_(True)  # 等价于x = torch.arange(4.0, requires_grad=True)
x.grad  # 默认值为None 

In [3]:
# 现在计算y
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
# 通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad  # y = 2x^2 y'=4x 每个点的导数分别为0，4，8，12

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

In [7]:
# 现在让我们计算x的另一个函数
# 在默认情况下，pytorch会积累梯度，我们需要清楚之前的值
x.grad.zero_()
y = x.sum()  # 向量x求和相当于向量x乘一个单位向量E，y'=E
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [8]:
# 深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和
# 对非标量调用backward需要传入一个gradient参数
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [9]:
# 将某些记录移动到记录的计算图之外
x.grad.zero_()
y = x * x
u = y.detach()  # u不再是一个关于x的函数，而是一个常数，值为x*x，可用于固定网络参数
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [10]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [14]:
# 即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），仍然可以计算得到变量的梯度
def f(a):
    b = a * 2
    while b.norm() < 1000:  # norm是L2范数
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)